# Week 2 - Objectives

1. Students will define classes and default, copy, and general constructors.
2. Students will write code that calls each one of the three types of constructors above.
3. Students will define destructors
4. Students will follow the RAII principles and define resources management classes and objects. 
5. Students will explain how the constructors are called when objects are passed as arguments to functions.
6. Students will explain which constructors are called when object data members are initilized.

## Reading:

[C++ Primer](https://cpp-primer.pages.dev/), Chapter 7:

1. Section 7.5 Constructors revisited (Attn!, C++11 rules)

[cppreference.com - constructors and member initializer lists](https://www.en.cppreference.com/w/cpp/language/initializer_list.html) - mentions the differences between the C++ standards.

In [1]:
// setup
#include <iostream>
#include <random>

# OOP - initialization: constructors
Since data members should be private, we need a **function** to initialize data members. This function is called a constructor. The following kinds of constructors are defined, depending on the number of arguments:
  - no arguments = default constructor
  - one argument which is an object of the same class = copy constructor
  - one or more arguments, not of the same class = general constructor
  
The constructor syntax is special, unlike that of declaring member functions.

In [2]:
class Rational {
    int num, den;
    
  public:
    // default
    Rational() {
        std::cout << "Rational()" << std::endl;
    }
    
    // copy
    Rational(const Rational & r) {
        std::cout << "Rational(const Rational&)" << std::endl;
    }
    
    // general constructor
    Rational(int n, int d) {
        std::cout << "Rational(int, int)" << std::endl;
    }
    
    // create a rational from an integer
    Rational(int value) {
        std::cout << "Rational(int)" << std::endl;
        num = value;
        den = 1;
    }
}

**Syntax for defining constructors:**
  - constructors are functions
  - they don't have a return type
  - same name as the class
  - the copy constructor must have its argument passed by reference.

In [3]:
// which constructors are called?
Rational r1;

Rational()


In [6]:
Rational r2 = r1;
Rational rr(r1);

Rational(const Rational&)
Rational(const Rational&)


In [7]:
Rational r3(1,2);

Rational(int, int)


In [8]:
Rational r4 = {2,3};

Rational(int, int)


In [10]:
Rational r5 = Rational(10,20);
Rational rrr(Rational(1,2));

Rational(int, int)
Rational(int, int)


In [11]:
void dummy(Rational r) {
    std::cout << "dummy(Rational)" << std::endl;
}

In [13]:
void dummy_ref(Rational & r) {
    std::cout << "dummy_ref(Rational &)" << std::endl;
}

In [12]:
dummy(r1);

Rational(const Rational&)
dummy(Rational)


In [14]:
dummy_ref(r1);

dummy_ref(Rational &)


In [15]:
r1 = r2;

In [4]:
Rational ri={3};
Rational rii(4);

Rational(int)
Rational(int)


**Syntax for calling constructors**

1. variable declaration with optional initialization.
2. Syntax for initialization in declaration can use function call syntax `()` or struct initializer list `{}` and insert the arguments for the constructor.
3. Constructors can be overloaded.

**Exercises**

1. Remove the debug/print statements from the rational class constructors and add useful initialization code. Remember that a rational cannot have a zero denominator.

2. Think about how to handle the situation when a programmer declares a rational with a zero denominator:
`Rational r(1,0);`.

**In-class exercise**

  1. Consider the following rational object `rat` declared inside the body of a for loop. What is the scope of the variable?
  2. Is a new `rat` object created every time the body of the loop executes?

In [ ]:
for (int i=0; i<10; i++) {
    Rational rat;
}

# The Yin and Yang of object initialization - destructors

When an object goes out of scope (dies), a special function called **destructor** is called. Goal: to free any resources that the object has acquired.

In [5]:
class Rational {
    int num, den;
    
  public:
    // default
    Rational() {
        std::cout << "Rational()" << std::endl;
    }
    
    // copy
    Rational(const Rational &r) {
        std::cout << "Rational(const Rational&)" << std::endl;
    }
    
    // general constructor
    Rational(int n, int d) {
        std::cout << "Rational(int, int)" << std::endl;
    }
    
    // destructor
    ~Rational() {
        std::cout << "~Rational()" << std::endl;
    }
}

**Syntax for defining destructors:**
1. class name with `~` in front of th name, no arguments, no return type. 

In [6]:
// a destructor is called when an object goes out of scope.
// So, we use a block statement to force this
{
    Rational r1;
    std::cout << "Do some work and we are done." << std::endl;
}

Rational()
Do some work and we are done.
~Rational()


**Syntax for calling destructors:**
1. Destructors are not called explicitly by the programmer.

# OOP - Resource Acquisition Is Initialization (RAII) #

Rule: create an object whenever your program must acquire a resource (memory, socket, file, mutex, etc.) 

  - acquire the resource in the constructor
  - release the resource in the destructor

### Example:

Suppose we need to execute `procedureA()`, but, to be successful, we must first acquire some resource `resourceR`. Of course, at the end, we must release `resourceR`. We define the following procedures that simulate these processes:

In [ ]:
void procedureA() {
    std::cout << "Procedure A" << std::endl;
}

In [ ]:
void acquireR() {
    std::cout << "Acquire R" << std::endl;
}

In [ ]:
void releaseR() {
    std::cout << "Release R" << std::endl;
}

Here is a valid sequence of operations:

In [ ]:
acquireR();
procedureA();
releaseR();

Here is an invalid sequence (procedure A will fail because it does not have resource R to succeed):

In [ ]:
procedureA();
acquireR();
releaseR();

How about the following sequences?

In [ ]:
acquireR();
releaseR();
procedureA();

In [ ]:
acquireR();
acquireR();
procedureA();
releaseR();

We can more safely manage resource R if we assign the responsibility for its management to an object:
  1. acquire the resource in the constructor
  2. release the resource in the destructor
  3. create a local resource management object every time we need the resource, for example in side a block statement.
  
This approach = RAII

In [ ]:
// resource manager class
class manageR {
    public:
        manageR() {
            acquireR();
        }
    
        ~manageR() {
            releaseR();
        }
}

In [ ]:
// using the resource
{
    manageR robj;
    procedureA();
}

**Advantages for RAII**
  1. One
  2. Two

**Exercise**: 
Suppose we expand the definition of resource R to include a number of units to be acquired. We also expand the definition of procedure A to require 10 units of R to complete. Procedure A also returns an integer between 0 and 10, representing the number of units of resource R needed by an additional procedure B. We define the following functions that simulate this process.

In [ ]:
int procedureA() {
    std::cout << "Procedure A runs and needs 10 R" << std::endl;
    std::random_device rd;
    std::minstd_rand gen(rd());                     
    std::uniform_int_distribution<> distr(0, 10);  // Range: [0, 10]

    int randomNumber = distr(gen);
    std::cout << "Procedure A returns " << randomNumber << std::endl;
    return randomNumber;
}

In [ ]:
void procedureB() {
    std::cout << "Procedure B" << std::endl;
}

In [ ]:
void acquireR(int units) {
    std::cout << "Acquire "<< units << " units of R" << std::endl;
}

In [ ]:
void releaseR(int units) {
    std::cout << "Release "<< units << " units of R" << std::endl;
}

Here is standard code to carry out procedures A and B, and manage resource R: 

In [ ]:
acquireR(10);
int eval_b = procedureA();
if (eval_b > 0) {
    acquireR(eval_b);
}
procedureB();
releaseR(10);
if (eval_b > 0) {
    releaseR(eval_b);
}

Use the RAII principles to execute procedures A and B and manage resource R appropriately.

# Constructors and initializer lists

How do we initialize an object of a class that has other objects as members? For example, suppose we define a linear polynomial $p(x) = ax + b$, where $a$ and $b$ are Rational?

In [ ]:
class Poly {
    Rational a,b;
    
  public:
    Poly(Rational coef1, Rational coef2);
}

In [ ]:
Poly::Poly(Rational coef1, Rational coef2) {
    a = coef1;
    b = coef2;
}

In [ ]:
Poly p(Rational{1,2}, Rational{15,13});

What do you notice?

We can avoid the unnecessary call to the default constructor (for $a$ and $b$) if we use an initializer list:

In [ ]:
Poly::Poly(Rational coef1, Rational coef2): a(coef1), b(coef2) {
    // nothing here
}

Suppose we add a second generic constructor that accepts the numerator and denominators of the $a$ and $b$ coefficients:

In [ ]:
class Poly {
    Rational a,b;
    
  public:
    Poly(Rational coef1, Rational coef2);
    
    Poly(int num1, int den1, int num2, int den2);
}

We can implement this constructor in a similar way as before,

In [ ]:
Poly::Poly(int num1, int den1, int num2, int den2) {
    a = Rational(num1, den1);
    b = Rational(num2, den2);
}

In [ ]:
Poly p(1,2,15,13);

... or we can use an initializer list:

In [ ]:
Poly::Poly(int num1, int den1, int num2, int den2): a(num1,den1), b(num1,den1) {
}

In [ ]:
Poly p(1,2,15,13);

**Exercise**:

Expand the definition of the `Turtle` class from `1-classes.ipynb` to contain a `CImg` object on which the turtle can draw. Use an initializer list to initialize the `CImg`.